In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
import numpy as np
from matplotlib import pyplot as plt

import sys
sys.path.append('../dependencies/')
import dataset_utils
import network_ec_bm as network
import utils
import zca

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

tf.random.set_seed(42)

tfd = tfp.distributions
tfk = tf.keras
tfkl = tf.keras.layers

train_set = 'celeb_a'
norm = None
mode = 'color'
pre = 'ec_bm/'

if mode == 'color':
    input_shape = (32, 32, 3)
    datasets = [
        'svhn_cropped',
        'cifar10',
        'celeb_a',
        'gtsrb',
        'compcars',
        'noise'
    ]
    num_filters = 64
elif mode == 'grayscale':
    input_shape = (32, 32, 1)
    datasets = [
    'mnist',
    'fashion_mnist',
    'emnist/letters',
    'sign_lang',
    'noise'
    ]
    num_filters = 32
    
mutation_rate = 0  
num_resnet = 2
num_hierarchies = 4
num_logistic_mix = 5
num_filters = num_filters
dropout_p = 0.3
learning_rate = 1e-3
use_weight_norm = True
epochs = 100
optimizer = tf.optimizers.Adam(learning_rate=learning_rate)

if norm is None:
    dir_str = 'original'
elif norm == 'pctile-5':
    dir_str = 'pctile-5'
elif norm == 'channelwhiten':
    dir_str = 'zca'
elif norm == 'zca_original':
    dir_str = 'zca_original'
elif norm == 'histeq':
    dir_str = 'histeq'
    
model_dir = '../saved_models/' + pre + dir_str + '/' + train_set + '/'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    
if norm == 'zca_original':
    zca_transform = zca.compute_zca(train_set)
else:
    zca_transform = None
    
ds_train, ds_val, _ = dataset_utils.get_dataset(
      train_set,
      32,
      mode,
      normalize=norm,
      dequantize=False,
      visible_dist='categorical',
      zca_transform=zca_transform,
      mutation_rate=mutation_rate
  )

dist = network.PixelCNN(
      image_shape=input_shape,
      num_resnet=num_resnet,
      num_hierarchies=num_hierarchies,
      num_filters=num_filters,
      num_logistic_mix=num_logistic_mix,
      dropout_p=dropout_p,
      use_weight_norm=use_weight_norm,
)

image_input = tfkl.Input(shape=input_shape)
log_prob = dist.log_prob(image_input)
model = tfk.Model(inputs=image_input, outputs=log_prob)
model.add_loss(-tf.reduce_mean(log_prob))
model.compile(optimizer=optimizer)

model.build([None] + list(input_shape))
# model.load_weights(model_dir+'weights')

# model.fit(
#         ds_train,
#         epochs=epochs,
#         validation_data=ds_val,
#         callbacks=[
#             utils.CNSModelCheckpoint(
#                 filepath=os.path.join(model_dir+'weights'),
#                 verbose=1, save_weights_only=True, save_best_only=True
#                 )
#         ],
#     )

model.load_weights(model_dir+'weights')

2022-05-08 14:27:51.946066: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-08 14:27:53.849907: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10407 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:41:00.0, compute capability: 6.1


In [2]:
import tqdm
from scipy.io import savemat
from scipy.io import loadmat

if train_set == 'emnist/letters':
    train_set = 'emnist_letters'
    
probs = {}

for dataset in datasets:
    
    _, _, ds_test = dataset_utils.get_dataset(
          dataset,
          1024,
          mode,
          normalize=norm,
          dequantize=False,
          visible_dist='categorical',
          zca_transform=zca_transform,
          mutation_rate=0
      )
    tmp = []
    for test_batch in tqdm.tqdm(ds_test):
        tmp.append(dist.log_prob(tf.cast(test_batch, tf.float32),
                                        training=False).numpy())

    tmp = np.expand_dims(np.concatenate(tmp, axis=0),axis=-1)
    tmp = np.array(tmp)
    
    probs[dataset] = tmp

save_dir = '../probs/' + pre + dir_str + '/'
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

savemat(save_dir + train_set + '.mat', probs)

  0%|                                                                                                                                                                                     | 0/26 [00:00<?, ?it/s]2022-05-08 14:28:49.684405: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-08 14:28:52.330203: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:05<00:00,  6.52s/it]
39it [04:24,  6.77s/it]
13it [02:22, 10.94s/it]
14it [04:11, 17.94s/it]
10it [00:13,  1.39s/it]
